In [1]:
import dataset
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#explain dataframe
tropes = dataset.TVTropesDataset.from_csv_files("TVTropesData")
tropes.lit_tropes
tropes.lit_tropes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679618 entries, 0 to 679617
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  679618 non-null  int64 
 1   Title       679618 non-null  object
 2   Trope       679618 non-null  object
 3   Example     662150 non-null  object
 4   trope_id    679618 non-null  object
 5   title_id    679618 non-null  object
dtypes: int64(1), object(5)
memory usage: 31.1+ MB


In [6]:
# statistics about Example column text data (avg number of chars etc.)
tropes.lit_tropes["Example"].describe()

count                                                662150
unique                                               554323
top        Fighting Fantasy, a wildly popular British se...
freq                                                     68
Name: Example, dtype: object

In [7]:
tropes.lit_tropes["Example"].value_counts()

Example
Fighting Fantasy, a wildly popular British series that includes simple Tabletop RPG elements, with dice as randomizers.\n The Warlock of Firetop Mountain by Steve Jackson and Ian Livingstone, the first installment and the one that started it all.\n The Citadel of Chaos by Steve Jackson, second installment of the series.\n The Forest of Doom by Ian Livingstone, third installment of the series.\n Starship Traveller by Steve Jackson, fourth installment of the series and first foray into Sci-Fi, heavily inspired by Star Trek.\n City of Thieves by Ian Livingstone, 5th installment of the series.\n Deathtrap Dungeon by Ian Livingstone, 6th installment of the series.\n Island of the Lizard King by Ian Livingstone, 7th installment of the series.\n Scorpion Swamp by Steve Jackson,noteNot the same Jackson who co-wrote Firetop Mountain and wrote Citadel, but the US game developer 8th installment of the series. Features unusual, multiple alignment-based paths and endings.\n Caverns of the S

In [8]:
# check sample of non unique rows
tropes.lit_tropes.nunique()

Unnamed: 0    679618
Title          15495
Trope          27229
Example       554323
trope_id       27229
title_id       15495
dtype: int64

In [12]:
# check sample of unique rows
unique_lit_tropes_texts = tropes.lit_tropes["Example"].unique()
print(unique_lit_tropes_texts[5:10])
print(unique_lit_tropes_texts.shape)

[' When Tyler is explaining to  Kelley  how   Mr. Click died , she says, "Are you kidding me?", "Are you serious?", and "You can\'t be serious! Are you kidding me?".'
 ' Implied by the cab driver in A Bad Day For Voodoo. When Tyler is in the cab, he tells the cab driver that he needs to borrow his phone to call his mom, and when he hands him the phone, he jokes that shes probably already in his contacts list.\n'
 ' In A Bad Day For Voodoo,  Esmeralda, the maker of the voodoo dolls, explains to Tyler and Kelley that Adam is this, and will one day defeat the hobgoblins. When Kelley questions this, she explains that only the chosen one could have made the voodoo dolls so powerful.\n'
 ' After  Mr. Click\'s  leg comes off, Tyler and Adam go to Tyler\'s house to watch the news, and the reporter announces that   Mr. Click is dead . Adam shouts, "No!".'
 ' Mr. Click. He gives  Tyler  an undeserved F, and announces to the entire class when a kid gets a bad grade.']
(554324,)


In [3]:
#checking if subsets
# Check if lit_goodreads_match is a subset of lit_tropes
lit_goodreads_match_subset = tropes.lit_goodreads_match[['trope_id', 'title_id']].merge(tropes.lit_tropes[['trope_id', 'title_id']], how='left', indicator=True)
print("Is lit_goodreads_match a subset of lit_tropes?")
print(lit_goodreads_match_subset['_merge'].eq('both').all())

# Check if film_imdb_match is a subset of film_tropes
film_imdb_match_subset = tropes.film_imdb_match[['trope_id', 'title_id']].merge(tropes.film_tropes[['trope_id', 'title_id']], how='left', indicator=True)
print("\nIs film_imdb_match a subset of film_tropes?")
print(film_imdb_match_subset['_merge'].eq('both').all())

Is lit_goodreads_match a subset of lit_tropes?
False

Is film_imdb_match a subset of film_tropes?
True


#

# Environment sanity check
Testing whether CUDA/torch installation.

In [1]:
from sentence_transformers import SentenceTransformer

# This model supports two prompts: "s2p_query" and "s2s_query" for sentence-to-passage and sentence-to-sentence tasks, respectively.
# They are defined in `config_sentence_transformers.json`
query_prompt_name = "s2p_query"
queries = [
    "What are some ways to reduce stress?",
    "What are the benefits of drinking green tea?",
]
# docs do not need any prompts
docs = [
    "There are many effective ways to reduce stress. Some common techniques include deep breathing, meditation, and physical activity. Engaging in hobbies, spending time in nature, and connecting with loved ones can also help alleviate stress. Additionally, setting boundaries, practicing self-care, and learning to say no can prevent stress from building up.",
    "Green tea has been consumed for centuries and is known for its potential health benefits. It contains antioxidants that may help protect the body against damage caused by free radicals. Regular consumption of green tea has been associated with improved heart health, enhanced cognitive function, and a reduced risk of certain types of cancer. The polyphenols in green tea may also have anti-inflammatory and weight loss properties.",
]

# ！The default dimension is 1024, if you need other dimensions, please clone the model and modify `modules.json` to replace `2_Dense_1024` with another dimension, e.g. `2_Dense_256` or `2_Dense_8192` !
model = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True).cuda()
query_embeddings = model.encode(queries, prompt_name=query_prompt_name)
doc_embeddings = model.encode(docs)
print(query_embeddings.shape, doc_embeddings.shape)
# (2, 1024) (2, 1024)

similarities = model.similarity(query_embeddings, doc_embeddings)
print(similarities)

d:\narana\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


(2, 1024) (2, 1024)
tensor([[0.8179, 0.2958],
        [0.3194, 0.7854]])
